# Day 7: Amplification Circuit

In [11]:
from collections import defaultdict
from itertools import permutations 
with open('input_day7.txt') as file:
     memory =[int(_) for _ in file.read().split(',')]

In [12]:
# memory

# Intcode Computer

In [13]:
class Amplifier:
    
    def __init__(self, memory, phase_setter):
        self.phase_setter = phase_setter
        self.mem = memory.copy()
        
    def computer(self, signal):
        mode = {
            '0' : lambda x, i: x[i],  # position mode
            '1' : lambda x, i: i     # immediate mode
        }
        mem = self.mem
        output = []
        i = 0
        while True:
            inst = str(mem[i]).zfill(5)
            A, B, C, _, opcode = inst
            if opcode == '9':
                return output[-1]
            elif opcode == '1':
                mem[mode[A](mem, i + 3)] = mem[mode[C](mem, i + 1)] + mem[mode[B](mem, i + 2)]
                i += 4
            elif opcode == '2':
                mem[mode[A](mem, i + 3)] = mem[mode[C](mem, i + 1)] * mem[mode[B](mem, i + 2)]
                i += 4
            elif opcode == '3':
                mem[mode[C](mem, i + 1)] = mem[mode[C](mem, i + 1)] = signal if i else self.phase_setter # firs value in mem is always 3
                i += 2
            elif opcode == '4':
                output.append(mem[mode[C](mem, i + 1)])
                i += 2
            elif opcode == '5':
                if mem[mode[C](mem, i + 1)]:
                    i =  mem[mode[B](mem, i + 2)]
                else:
                    i += 3
            elif opcode == '6':
                if not mem[mode[C](mem, i + 1)]:
                    i =  mem[mode[B](mem, i + 2)]
                else:
                    i += 3
            elif opcode == '7':
                if mem[mode[C](mem, i + 1)] < mem[mode[B](mem, i + 2)]:
                    mem[mode[A](mem, i + 3)] = 1
                else:
                    mem[mode[A](mem, i + 3)] =  0
                i += 4
            elif opcode == '8':
                if mem[mode[C](mem, i + 1)] == mem[mode[B](mem, i + 2)]:
                    mem[mode[A](mem, i + 3)] = 1
                else:
                    mem[mode[A](mem, i + 3)] =  0
                i += 4
            else:
                raise IndexError(f'Index {i} unacceptable')

# Part 1

In [18]:
def find_max_signal_to_thruster(memory):
    signals_to_thruster = []
    for combination in permutations(range(5)):
        print(combination)
        amplifiers = [Amplifier(memory, phase_setter) for phase_setter in combination]
        sig = 0
        for i in range(5):
            sig = amplifiers[i].computer(sig)
            print(sig)
        signals_to_thruster.append(sig)
    return max(signals_to_thruster)
    

In [19]:
find_max_signal_to_thruster(memory)

(0, 1, 2, 3, 4)
33
103
2502
2507
10028
(0, 1, 2, 4, 3)
33
103
2502
10008
10013
(0, 1, 3, 2, 4)
33
103
108
2622
10488
(0, 1, 3, 4, 2)
33
103
108
432
10398
(0, 1, 4, 2, 3)
33
103
412
9918
9923
(0, 1, 4, 3, 2)
33
103
412
417
10038
(0, 2, 1, 3, 4)
33
822
2470
2475
9900
(0, 2, 1, 4, 3)
33
822
2470
9880
9885
(0, 2, 3, 1, 4)
33
822
827
2485
9940
(0, 2, 3, 4, 1)
33
822
827
3308
9928
(0, 2, 4, 1, 3)
33
822
3288
9868
9873
(0, 2, 4, 3, 1)
33
822
3288
3293
9883
(0, 3, 1, 2, 4)
33
38
118
2862
11448
(0, 3, 1, 4, 2)
33
38
118
472
11358
(0, 3, 2, 1, 4)
33
38
942
2830
11320
(0, 3, 2, 4, 1)
33
38
942
3768
11308
(0, 3, 4, 1, 2)
33
38
152
460
11070
(0, 3, 4, 2, 1)
33
38
152
3678
11038
(0, 4, 1, 2, 3)
33
132
400
9630
9635
(0, 4, 1, 3, 2)
33
132
400
405
9750
(0, 4, 2, 1, 3)
33
132
3198
9598
9603
(0, 4, 2, 3, 1)
33
132
3198
3203
9613
(0, 4, 3, 1, 2)
33
132
137
415
9990
(0, 4, 3, 2, 1)
33
132
137
3318
9958
(1, 0, 2, 3, 4)
4
129
3126
3131
12524
(1, 0, 2, 4, 3)
4
129
3126
12504
12509
(1, 0, 3, 2, 4)
4
129
134
3

47064

# Tests

In [8]:
memory =  [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
phase = [4,3,2,1,0]
amplifiers = [Amplifier(memory, phase_setter) for phase_setter in [4,3,2,1,0]]
sig = 0
for i in range(5):
    sig = amplifiers[i].computer(sig)
    print(sig)
assert sig  == 43210

4
43
432
4321
43210


In [9]:
memory =  [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
amplifiers = [Amplifier(memory, phase_setter) for phase_setter in [0,1,2,3,4]]
sig = 0
for i in range(5):
    sig = amplifiers[i].computer(sig)
    print(sig)
assert sig  == 54321

5
54
543
5432
54321


In [10]:
memory =  [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
amplifiers = [Amplifier(memory, phase_setter) for phase_setter in [1,0,4,3,2]]
sig = 0
for i in range(5):
    sig = amplifiers[i].computer(sig)
    print(sig)
assert sig  == 65210

6
65
652
6521
65210
